In [ ]:
import pandas as pd
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go

#chartTemplate = 'plotly_dark'
chartTemplate = 'plotly_white'

In [ ]:
# Datos de Cuba de https://ourworldindata.org/covid-cases

#casos = pd.read_csv("owid-covid-data.csv")
casos = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

cuba0 = casos[casos["location"] == "Cuba"][["date","new_cases"]]
cuba = cuba0.reset_index().rename(columns={'date': 'Date','new_cases':'N'}).sort_values(by='Date', ascending=True)
cuba

,index,Date,N
0,26642,2020-03-12,3.0
1,26643,2020-03-13,1.0
2,26644,2020-03-14,0.0
3,26645,2020-03-15,0.0
4,26646,2020-03-16,0.0
...,...,...,...
560,27202,2021-09-23,6935.0
561,27203,2021-09-24,7695.0
562,27204,2021-09-25,7513.0
563,27205,2021-09-26,6673.0


In [ ]:
cfg = pd.read_csv("../covid19-casos-cfg2.csv", delimiter=';')
#cases = pd.read_csv("../covid19-casos.csv")

#cases_cfg = cases[cases["provincia"] == "Cienfuegos"]

cfg.rename(columns={'fecha': 'Date', 'totales': 'test', 'positivos': 'N'}, inplace=True)

cfg

,Date,test,N
0,17/01/2021,12,2
1,18/01/2021,1566,24
2,19/01/2021,1000,37
3,20/01/2021,90,3
4,21/01/2021,638,5
...,...,...,...
245,23/09/2021,300,180
246,24/09/2021,259,102
247,25/09/2021,287,168
248,26/09/2021,183,129


In [ ]:

# calculate case velocity
#cuba["vel"] = cuba.loc[:, cuba.columns != 'Date'].diff()
cuba["vel"] = cuba.loc[:, 'N'].diff()
cfg["vel"] = cfg.loc[:, 'N'].diff()

# calculate case acceleration
cuba["acc"] = cuba[["Date", "vel"]].loc[:, 'vel'].diff()
cfg["acc"] = cfg[["Date", "vel"]].loc[:, 'vel'].diff()

# calculate means - 14 días - 2 semanas
# usando rolling
avg = 14
cfg["accelMeans14"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cfg["accelMeans7"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = 7).mean()
cfg["accelMeans3"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = 3).mean()
cuba["accelMeans14"] = cuba[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cuba["accelMeans7"] = cuba[["Date", "vel"]].loc[:, "vel"].rolling(window = 7).mean()
cuba["accelMeans3"] = cuba[["Date", "vel"]].loc[:, "vel"].rolling(window = 3).mean()

# con EWM
# cfg["accelMeans5"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Datetime'].ewm(span=5).mean()

cfg

,Date,test,N,vel,acc,accelMeans14,accelMeans7,accelMeans3
0,17/01/2021,12,2,NaN,NaN,NaN,NaN,NaN
1,18/01/2021,1566,24,22.0,NaN,NaN,NaN,NaN
2,19/01/2021,1000,37,13.0,-9.0,NaN,NaN,NaN
3,20/01/2021,90,3,-34.0,-47.0,NaN,NaN,0.333333
4,21/01/2021,638,5,2.0,36.0,NaN,NaN,-6.333333
...,...,...,...,...,...,...,...,...
245,23/09/2021,300,180,-56.0,-77.0,-15.071429,-27.714286,19.333333
246,24/09/2021,259,102,-78.0,-22.0,-17.785714,-25.142857,-37.666667
247,25/09/2021,287,168,66.0,144.0,-11.785714,-14.142857,-22.666667
248,26/09/2021,183,129,-39.0,-105.0,-4.071429,-9.857143,-17.000000


In [ ]:
fig = px.line(cfg, x="Date", y="N", template=chartTemplate)
fig.update_layout({"title": 'Casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

#  ##--

# CUBA

## Análisis de Cuba

#  ##--

In [ ]:
fig = px.line(cuba, x="Date", y="N", template=chartTemplate)

fig.update_layout({"title": 'Casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [ ]:
pos = cuba[cuba['accelMeans7'] >= 0]
neg = cuba[cuba['accelMeans7'] < 0]

fig = px.line(cuba, x="Date", y="accelMeans14", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})
fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans7'], name='Promedio de 14 días', opacity=0.5))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans7'], name='Promedio de 14 días', opacity=0.5))

fig.show()

In [ ]:
bars = 'accelMeans7'
#bars = 'accelMeans3'
#bars = 'acc'
#bars = 'vel'
cfg['color'] = cfg.apply(lambda x: 'green' if x['accelMeans7'] <= 0 else 'red', axis=1)

fig = go.Figure()

### Traces
# Line
fig.add_trace(go.Scatter( #type= 'area',
    x = cfg['Date'],
    y = cfg['accelMeans14'], #cfg[cfg['accelMeans14']<0]['accelMeans14'], #y = ,
    name = 'Aceleración 14d',
    line = dict(
        color = 'blue', # cuba['color'],
        width = 1.5)
))
# Bars
fig.add_trace(go.Bar(
    x=cfg['Date'], 
    y=cfg[bars], 
    name='Aceleración 3d', 
    opacity=0.5,
    marker=dict(color=cfg['color']),
    yaxis = 'y2'
))

# First apply general tamplate
fig.update_layout(template = chartTemplate)

### New spec Layout
layout = {'title': 'Aceleración de casos - promedio móvil 14 días (línea) - 3 días (barras) - Cuba',
             "width" : 800,
            "height" : 500,
            'font' : {
                'family': "Arial",
                'size': 12,
                'color': 'rgb(96, 96, 96)' #"gray"
            },
            'xaxis': {
                'dtick': '30',
                'tickformat': '%d %B',
                'showline': True,
                'showgrid': False,
                'showticklabels': True,
                'linecolor': 'rgb(204, 204, 204)',
                'linewidth': 2,
                'ticks': 'outside',
                'tickfont': dict(
                    family='Arial',
                    size=12,
                    color='rgb(82, 82, 82)',
        ),
            },
            'yaxis': {
                    'zerolinecolor': 'black',
                    'zerolinewidth': 1,
                    'rangemode': 'tozero',
                    #'gridcolor': 'gray',
                    #'gridwidth': 0.1
                    #'range': [150,-150]
            },
            'yaxis2': {
                    #'title': '3 dias',
                    'overlaying': 'y',
                    'side': 'right',
                    #'range': [250,-250],
                    'zerolinecolor': 'black',
                    'zerolinewidth': 1,
                },
            #'legend_title': "Legend Title",
            'legend': { 
                    'x': .1,
                    'y': .98
            }
         }
fig.update_layout(layout)
fig.write_image("figure.png")
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=331fe212-3056-4b5e-ab8f-59cd76bdab29' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>